In [ ]:
%store -r
import time
import os,json
import requests, operator
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo, DataType
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitorAlert, TimeZone

In [ ]:
MONITOR_NAME = d3_config['MONITOR_NAME']
DATA_SOURCE = d3_config['DATA_SOURCE']
INPUT_TRAIN_TYPE = d3_config['INPUT_TRAIN_TYPE']
DKUBEUSERNAME = d3_config['DKUBEUSERNAME']
TOKEN = d3_config['TOKEN']
DKUBE_URL = d3_config['DKUBE_URL']
DKUBE_BASE_DATASET = d3_config['DKUBE_BASE_DATASET']
MODEL_NAME = d3_config['MODEL_NAME']
RETRAINING_DATASET = d3_config['RETRAINING_DATASET']
RUN_FREQUENCY = d3_config['RUN_FREQUENCY']
USE_REMOTE_DEPLOYMENT = d3_config['USE_REMOTE_DEPLOYMENT']

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)

In [ ]:
def wait_for_deployment_running(deployment_id):
    status = None
    inference_url, inference = None, None
    while True:
        data = api.get_deployment(deployment_id)
        status = data.data.inferenceservice_deployment.parameters.generated.status.state
        inference = data.data.inferenceservice_deployment.parameters.inference
        inference_url = data.data.inferenceservice_deployment.parameters.generated.details.serving.servingurl
        if status == "RUNNING":
            break
        print("waiting for deployment to be running")
        time.sleep(api.wait_interval)
    return inference, inference_url

def get_dataset_version(username, dataset_name, version):
    dataset_versions = api.get_dataset_versions(username, dataset_name)
    versions = []
    for each_version in dataset_versions:
        if each_version["version"]["name"] == version:
            uuid = each_version["version"]["uuid"]
            return f"{version}:{uuid}"
        else:
            versions.append(each_version["version"]["name"])
    return f"dataset version {version} not found, available version are {versions}"

### Importing deployment

In [ ]:
if USE_REMOTE_DEPLOYMENT:
    api.import_deployment(name=MONITOR_NAME)
while True:
    DEPLOYMENT_ID = api.get_deployment_id(name=MONITOR_NAME)
    if DEPLOYMENT_ID:
        break
    print("waiting for deployment to come up")
    time.sleep(api.wait_interval)
if not USE_REMOTE_DEPLOYMENT:
    inference, INFERENCE_URL = wait_for_deployment_running(DEPLOYMENT_ID)
    print("Inference is up at URL: ", INFERENCE_URL)
%store DEPLOYMENT_ID

In [ ]:
text_file = open("transform-data.py", "r")
script = text_file.read()
text_file.close()

with open('thresholds.json') as f:
    thresholds = json.load(f)

### Model Monitor

In [ ]:
mm=DkubeModelmonitor(deployemnt_id = DEPLOYMENT_ID)

In [ ]:
mm.update_modelmonitor_basics(model_type=ModelType.Regression.value, 
                               input_data_type=DataType.Tabular.value,
                               data_timezone=TimeZone.UTC.value)

In [ ]:
mm.add_thresholds(thresholds=thresholds)

## Health Monitoring

In [ ]:
# checking if imported cluster has cluster added 
# Or is it a local deployemnt.
deployment = api.get_deployment(DEPLOYMENT_ID)
if ((deployment.data.imported_deployment and deployment.data.imported_deployment.cluster)
    or
    not deployment.data.imported_deployment):
    mm.update_deployment_monitoring_details(enabled=True, frequency=1)

### Add Drift monitoring details

In [ ]:
mm.update_drift_monitoring_details(enabled=True,frequency=5,algorithm='auto')

### Training Details

In [ ]:
if DATA_SOURCE in ['local','aws-s3', 'minio']:
    training_data = f'{DKUBEUSERNAME}:{DKUBE_BASE_DATASET}'
    train_data_version = get_dataset_version(DKUBEUSERNAME,
                                            DKUBE_BASE_DATASET, "v1")
    prediction_data = f"{DKUBEUSERNAME}:{MONITOR_NAME}-predict"
    labelled_data = f"{DKUBEUSERNAME}:{MONITOR_NAME}-groundtruth"

if DATA_SOURCE == 'local':
    predict_data_version = get_dataset_version(DKUBEUSERNAME,
                                            MONITOR_NAME+'-predict', "v1")
    labelled_data_version = get_dataset_version(DKUBEUSERNAME,
                                            MONITOR_NAME+'-groundtruth', "v1")

predict_data_format = str(DatasetFormat.Tabular)

if DATA_SOURCE == "sql":
    training_data = f'{DKUBEUSERNAME}:{DKUBE_BASE_DATASET}'

### Add Training, Predict and labelled Dataset

In [ ]:
if DATA_SOURCE == 'sql':
    mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),sql_query="select * from insurance",transformer_script = script)
    mm.add_datasources(data_class=str(DatasetClass.Predict),name=training_data,data_format=str(DatasetFormat.Tabular),sql_query="select * from insurance_predict",date_suffix="yyyy/mm/dd/hh")
    mm.add_datasources(data_class=str(DatasetClass.Labelled),name=training_data,data_format=str(DatasetFormat.Tabular),sql_query="select * from insurance_gt",predict_col="charges",groundtruth_col="GT_target",timestamp_col="timestamp")

if DATA_SOURCE == 'local':
    mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),version=train_data_version,transformer_script = script)
    mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,data_format=str(DatasetFormat.Tabular),version=predict_data_version,date_suffix="none")
    mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,data_format=str(DatasetFormat.Tabular),version=labelled_data_version,predict_col="charges",groundtruth_col="GT_target",timestamp_col="timestamp")

if DATA_SOURCE in ['aws-s3', 'minio']:
    mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,data_format=str(DatasetFormat.Tabular),version=train_data_version,transformer_script = script)
    mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,data_format=predict_data_format,date_suffix="yyyy/mm/dd/hh")
    mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,data_format=str(DatasetFormat.Tabular),predict_col="charges",groundtruth_col="GT_target",timestamp_col="timestamp")

### Performance Monitoring

In [ ]:
mm.update_performance_monitoring_details(enabled=True,source_type="labelled_data",frequency=RUN_FREQUENCY)

### Create Model monitor

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

### Extracting id of the Model Monitor

In [ ]:
# The function below can be used to fetch model monitor ID by name.
# The monitor id will be same as deployment id.
# id = api.modelmonitor_get_id(MONITOR_NAME)
id = DEPLOYMENT_ID
print(id)

### Schema update

In [ ]:
api.modelmonitor_update_schema(id,label='charges',schema_class='continuous',schema_type="prediction_output",selected=False)
api.modelmonitor_update_schema(id,label='unique_id',schema_class='continuous',schema_type="row_id",selected=False)
api.modelmonitor_update_schema(id,label='timestamp',schema_class='continuous',schema_type="timestamp",selected=False)

## age and bmi to continuous
api.modelmonitor_update_schema(id,label='age',schema_class='continuous',schema_type='input_feature',selected=True)
api.modelmonitor_update_schema(id,label='bmi',schema_class='continuous',schema_type='input_feature',selected=True)

## select these features
api.modelmonitor_update_schema(id,label='sex',schema_type='input_feature',schema_class='categorical',selected=True)
api.modelmonitor_update_schema(id,label='children',schema_type='input_feature',schema_class='categorical',selected=True)
api.modelmonitor_update_schema(id,label='smoker',schema_type='input_feature',schema_class='categorical',selected=True)
api.modelmonitor_update_schema(id,label='region',schema_type='input_feature',schema_class='categorical',selected=True)

### Add alerts

#### Deployment Health Alert

In [ ]:
if ((deployment.data.imported_deployment and deployment.data.imported_deployment.cluster)
    or
    not deployment.data.imported_deployment):
    alert = DkubeModelmonitorAlert(name='latency_alert', alert_class = 'deployment_health')
    alert.add_alert_condition(metric='latency_avg',threshold=300, op=operator.gt)
    api.modelmonitor_add_alert(id,alert)

#### Feature Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='age_alert', alert_class = 'feature_drift')
alert.add_alert_condition(feature='age',threshold=0.1, op=operator.gt)
api.modelmonitor_add_alert(id,alert)

#### Performance Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='mae_alert', alert_class = 'performance_decay')
alert.add_alert_condition(metric='mae',threshold=2000, op=operator.gt)
api.modelmonitor_add_alert(id,alert)

### Start the model monitor

In [ ]:
api.modelmonitor_start(id)

### Retraining / Rebaselining Model Monitor

In [ ]:
%store -r
id = api.modelmonitor_get_id(MONITOR_NAME)

if INPUT_TRAIN_TYPE == 'retraining':
    api.modelmonitor_stop(id)
    
    training_data = f'{RETRAINING_DATASET}:'+DKUBEUSERNAME
    data_dict = api.get_dataset_versions(DKUBEUSERNAME,RETRAINING_DATASET)[0]['version']
    train_data_version = data_dict['name']+":"+data_dict['uuid']
    
    mm=DkubeModelmonitor(name=MONITOR_NAME)
    mm.update_datasources(name=training_data,data_class=str(DatasetClass.Train),version=train_data_version)
    api.modelmonitor_update(id,mm)
    
    
    ### Start the model monitor
    api.modelmonitor_start(id)

### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(id)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(id)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(id) 